<a href="https://colab.research.google.com/github/Ira-a02/Algorithm/blob/main/ACO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
import random

In [36]:
def get_eta(task_times):
    return 1 / np.array(task_times)

In [37]:
def calculate_probabilities(pheromones, eta, alpha, beta):
    probabilities = (pheromones ** alpha) * (eta ** beta)
    probabilities_sum = probabilities.sum()

    if probabilities_sum > 0:
        return probabilities / probabilities_sum
    else:
        return np.ones_like(probabilities)

In [38]:
def update_pheromones(pheromones, solutions, delays, rho):
    pheromones = (1 - rho)*pheromones
    for solution, delay in zip(solutions, delays):
        for task in solution:
            pheromones[task] = pheromones[task] + 1 / delay
    return pheromones

In [39]:
def get_solution(num_tasks, pheromones, eta, alpha, beta):
    visited = np.zeros(num_tasks, dtype=bool)
    solution = []

    for i in range(num_tasks):
        unvisited_indices = np.where(~visited)[0]
        unvisited_eta = eta[unvisited_indices]
        unvisited_pheromones = pheromones[unvisited_indices]
        probabilities = calculate_probabilities(unvisited_pheromones, unvisited_eta, alpha, beta)


        next_task_index = np.random.choice(unvisited_indices, p=probabilities)
        solution.append(next_task_index)
        visited[next_task_index] = True

    return solution

In [41]:
def ACO(task_times, deadlines, ants, iter, alpha, beta, rho):
    num_tasks = len(task_times)
    pheromones = np.ones(num_tasks)
    best_solution = None
    best_delay = float('inf')
    eta = get_eta(task_times)

    for i in range(iter):
        solutions = []
        delays = []

        for ant in range(ants):
            solution = get_solution(num_tasks, pheromones, eta, alpha, beta)
            completion_times = np.zeros(num_tasks)
            total_delay = 0

            for i, task in enumerate(solution):
                if i == 0:
                    completion_times[task] = task_times[task]
                else:
                    completion_times[task] = completion_times[solution[i-1]] + task_times[task]

                total_delay += max(0, completion_times[task] - deadlines[task])

            solutions.append(solution)
            delays.append(total_delay)

            if total_delay <= best_delay:
                best_delay = total_delay
                best_solution = solution

        pheromones = update_pheromones(pheromones, solutions, delays, rho)

    return best_solution, best_delay

In [43]:
n = 5  # количество задач
ants=10
iter=200
alpha=1.0
beta=4.0
rho=0.4
#task_times = np.array([3, 2, 5, 1, 4])
#deadlines = np.array([5, 6, 7, 3, 8])
task_times = np.random.randint(1, 10, size=n)
deadlines = np.random.randint(1, 20, size=n)

best_solution, best_delay = ACO(task_times, deadlines, ants, iter, alpha, beta, rho)

print("Лучшее решение:", best_solution)
print("Суммарное запаздывание:", best_delay)
print("Время выполнения задач:", task_times)
print("Дедлайны задач:", deadlines)

Лучшее решение: [np.int64(0), np.int64(4), np.int64(2), np.int64(3), np.int64(1)]
Суммарное запаздывание: 6.0
Время выполнения задач: [1 6 2 5 9]
Дедлайны задач: [10 17 19 17 10]
